In [13]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda, RunnableSequence, RunnablePassthrough
from langchain_core.runnables import RunnableSequence, RunnablePassthrough
from langchain_core.runnables import RunnableBranch

from dotenv import load_dotenv
import os

load_dotenv()
api_key_01 = os.getenv("HUGGINGFACE_API_KEY")
api_key_02 = os.getenv("GOOGLE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key_01,
    temperature=0.1 # Lower temperature for better JSON generation
)

model_01 = ChatHuggingFace(llm=llm) 

str_parser = StrOutputParser()

prompt_01 = PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=["topic"]
)

prompt_02 = PromptTemplate(
    template="Summarize the following report: {report}",
    input_variables=["report"]
)

report_chain = prompt_01 | model_01 | str_parser


branch_chain = RunnableBranch(
    (lambda x: len(x.split())>20, RunnableSequence(prompt_02 | model_01 | str_parser)),
    (RunnablePassthrough()),
)

final_chain = report_chain | branch_chain


result = final_chain.invoke({"topic": "Pakistan's Economy VS India's Economy"})

print(result)

**Economic Comparison Report: Pakistan vs India**

This report compares the economies of Pakistan and India, two neighboring countries in South Asia with a long-standing rivalry. The key findings are:

**Key Indicators:**

- **GDP (Nominal):** India's GDP is significantly higher at $3.53 trillion compared to Pakistan's $276 billion.
- **GDP (PPP):** India's GDP (PPP) is also higher at $11.38 trillion compared to Pakistan's $1.24 trillion.
- **GDP Growth Rate:** India's GDP growth rate is higher at 7.2% compared to Pakistan's 3.9%.
- **Inflation Rate:** Pakistan's inflation rate is higher at 12.1% compared to India's 6.5%.
- **Unemployment Rate:** India's unemployment rate is higher at 7.4% compared to Pakistan's 5.8%.
- **Foreign Exchange Reserves:** India has significantly higher foreign exchange reserves at $540 billion compared to Pakistan's $8.4 billion.
- **Trade Balance:** Both countries have a trade deficit, with India's being slightly higher at -$20.4 billion compared to Pakist

In [14]:
final_chain.get_graph().print_ascii()

  +-------------+    
  | PromptInput |    
  +-------------+    
          *          
          *          
          *          
+----------------+   
| PromptTemplate |   
+----------------+   
          *          
          *          
          *          
+-----------------+  
| ChatHuggingFace |  
+-----------------+  
          *          
          *          
          *          
+-----------------+  
| StrOutputParser |  
+-----------------+  
          *          
          *          
          *          
    +--------+       
    | Branch |       
    +--------+       
          *          
          *          
          *          
  +--------------+   
  | BranchOutput |   
  +--------------+   
